In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/en.openfoodfacts.org.products']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.tsv', file_base + '.scaled.tsv')
      continue
    df_to_scale = pd.read_csv(file_base + '.tsv', delimiter='\t')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.tsv', sep='\t')

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
#imports 

# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])
import numpy as np
# ALEX: remove plotting, ML code
# import matplotlib.pyplot as plt
# import seaborn as sns

# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC, LinearSVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB

In [2]:
food_df = pd.read_csv("./input/en.openfoodfacts.org.products.scaled.tsv", delimiter='\t');
#food_df = pd.read_csv("FoodFacts.csv")

food_df.head()

/tmp/ipykernel_53954/1299057885.py:1: DtypeWarning: Columns (0,3,5,27,36) have mixed types. Specify dtype option on import or set low_memory=False.
  food_df = pd.read_csv("./input/en.openfoodfacts.org.products.tsv", delimiter='\t');


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24600,http://world-en.openfoodfacts.org/product/0000...,date-limite-app,1434530704,2015-06-17T08:45:04Z,1434535914,2015-06-17T10:11:54Z,Filet de bœuf,NaN,2.46 kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27083,http://world-en.openfoodfacts.org/product/0000...,canieatthis-app,1472223782,2016-08-26T15:03:02Z,1472223782,2016-08-26T15:03:02Z,Marks % Spencer 2 Blueberry Muffins,NaN,230g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27205,http://world-en.openfoodfacts.org/product/0000...,tacinte,1458238630,2016-03-17T18:17:10Z,1458238638,2016-03-17T18:17:18Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36252,http://world-en.openfoodfacts.org/product/0000...,tacinte,1422221701,2015-01-25T21:35:01Z,1422221855,2015-01-25T21:37:35Z,Lion Peanut x2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
food_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134754 entries, 0 to 134753
Columns: 161 entries, code to water-hardness_100g
dtypes: float64(105), object(56)
memory usage: 165.5+ MB


In [4]:
#split train/validate/test set

train, validate, test = np.split(food_df.sample(frac=1), [int(.6*len(food_df)), int(.8*len(food_df))])
print("Train: %s\nTest: %s\nValidate: %s" % (train.shape, test.shape, validate.shape))

Train: (80852, 161)
Test: (26951, 161)
Validate: (26951, 161)


In [5]:
#still working on this so I wanted to practice with a small dataset
df = validate
a, b = np.split(df.sample(frac=1), [int(.1*len(df))])
a.shape

(2695, 161)

In [6]:
a.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
133109,9300617032618,http://world-en.openfoodfacts.org/product/9300...,foodorigins,1458615780,2016-03-22T03:03:00Z,1458615805,2016-03-22T03:03:25Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21988,3038359004872,http://world-en.openfoodfacts.org/product/3038...,openfoodfacts-contributors,1481708965,2016-12-14T09:49:25Z,1487350640,2017-02-17T16:57:20Z,Lentilles faciles,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,-5.0,NaN,NaN
102924,6920354814334,http://world-en.openfoodfacts.org/product/6920...,openfoodfacts-contributors,1442734711,2015-09-20T07:38:31Z,1466591854,2016-06-22T10:37:34Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8086,0894455000490,http://world-en.openfoodfacts.org/product/0894...,openfoodfacts-contributors,1427840682,2015-03-31T22:24:42Z,1439141754,2015-08-09T17:35:54Z,Chocolate hazelnut butter,NaN,1 lbs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58098,3396411221507,http://world-en.openfoodfacts.org/product/3396...,openfoodfacts-contributors,1473873045,2016-09-14T17:10:45Z,1473883965,2016-09-14T20:12:45Z,6 mini moelleux au chocolat,NaN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,18.0,18.0,NaN,NaN


In [7]:
#clean data
#remove from entry data that has empty values in the energy
a = a[~pd.isnull(a['energy_100g'])]
a.drop(["code", "url", "creator", "created_t", "created_datetime", "last_modified_t", "last_modified_datetime"], axis=1)
a.describe()

,no_nutriments,additives_n,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,nutrition_grade_uk,energy_100g,energy-from-fat_100g,fat_100g,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
count,0.0,1215.000000,1215.000000,0.0,1215.000000,0.0,0.0,1512.000000,15.000000,1376.000000,...,0.0,31.000000,1.0,20.000000,0.0,7.000000,1331.000000,1331.000000,0.0,0.0
mean,NaN,1.879835,0.072428,NaN,0.130864,NaN,NaN,1133.325086,711.140000,14.177907,...,NaN,42.698387,15.0,55.695000,NaN,328.414286,8.752817,8.603306,NaN,NaN
std,NaN,2.524094,0.262459,NaN,0.399951,NaN,NaN,796.887471,904.041392,18.484408,...,NaN,24.166011,NaN,21.613044,NaN,393.887943,9.157423,9.550363,NaN,NaN
min,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,...,NaN,4.800000,15.0,19.900000,NaN,5.000000,-13.000000,-13.000000,NaN,NaN
25%,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,415.750000,64.150000,1.000000,...,NaN,26.500000,15.0,32.750000,NaN,29.800000,1.000000,0.000000,NaN,NaN
50%,NaN,1.000000,0.000000,NaN,0.000000,NaN,NaN,1051.000000,280.000000,6.700000,...,NaN,45.000000,15.0,59.000000,NaN,335.000000,9.000000,8.000000,NaN,NaN
75%,NaN,3.000000,0.000000,NaN,0.000000,NaN,NaN,1664.250000,980.500000,22.600000,...,NaN,52.000000,15.0,70.750000,NaN,386.150000,16.000000,17.000000,NaN,NaN
max,NaN,16.000000,2.000000,NaN,3.000000,NaN,NaN,3700.000000,2990.000000,100.000000,...,NaN,100.000000,15.0,88.000000,NaN,1127.000000,31.000000,31.000000,NaN,NaN


In [8]:
## analyze data 
# which features are available in the dataset? 
print(a.columns.values)

['code' 'url' 'creator' 'created_t' 'created_datetime' 'last_modified_t'
 'last_modified_datetime' 'product_name' 'generic_name' 'quantity'
 'packaging' 'packaging_tags' 'brands' 'brands_tags' 'categories'
 'categories_tags' 'categories_en' 'origins' 'origins_tags'
 'manufacturing_places' 'manufacturing_places_tags' 'labels' 'labels_tags'
 'labels_en' 'emb_codes' 'emb_codes_tags' 'first_packaging_code_geo'
 'cities' 'cities_tags' 'purchase_places' 'stores' 'countries'
 'countries_tags' 'countries_en' 'ingredients_text' 'allergens'
 'allergens_en' 'traces' 'traces_tags' 'traces_en' 'serving_size'
 'no_nutriments' 'additives_n' 'additives' 'additives_tags' 'additives_en'
 'ingredients_from_palm_oil_n' 'ingredients_from_palm_oil'
 'ingredients_from_palm_oil_tags'
 'ingredients_that_may_be_from_palm_oil_n'
 'ingredients_that_may_be_from_palm_oil'
 'ingredients_that_may_be_from_palm_oil_tags' 'nutrition_grade_uk'
 'nutrition_grade_fr' 'pnns_groups_1' 'pnns_groups_2' 'states'
 'states_tags' 

In [9]:
a.describe(exclude=['O'])

,no_nutriments,additives_n,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,nutrition_grade_uk,energy_100g,energy-from-fat_100g,fat_100g,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
count,0.0,1215.000000,1215.000000,0.0,1215.000000,0.0,0.0,1512.000000,15.000000,1376.000000,...,0.0,31.000000,1.0,20.000000,0.0,7.000000,1331.000000,1331.000000,0.0,0.0
mean,NaN,1.879835,0.072428,NaN,0.130864,NaN,NaN,1133.325086,711.140000,14.177907,...,NaN,42.698387,15.0,55.695000,NaN,328.414286,8.752817,8.603306,NaN,NaN
std,NaN,2.524094,0.262459,NaN,0.399951,NaN,NaN,796.887471,904.041392,18.484408,...,NaN,24.166011,NaN,21.613044,NaN,393.887943,9.157423,9.550363,NaN,NaN
min,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,...,NaN,4.800000,15.0,19.900000,NaN,5.000000,-13.000000,-13.000000,NaN,NaN
25%,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,415.750000,64.150000,1.000000,...,NaN,26.500000,15.0,32.750000,NaN,29.800000,1.000000,0.000000,NaN,NaN
50%,NaN,1.000000,0.000000,NaN,0.000000,NaN,NaN,1051.000000,280.000000,6.700000,...,NaN,45.000000,15.0,59.000000,NaN,335.000000,9.000000,8.000000,NaN,NaN
75%,NaN,3.000000,0.000000,NaN,0.000000,NaN,NaN,1664.250000,980.500000,22.600000,...,NaN,52.000000,15.0,70.750000,NaN,386.150000,16.000000,17.000000,NaN,NaN
max,NaN,16.000000,2.000000,NaN,3.000000,NaN,NaN,3700.000000,2990.000000,100.000000,...,NaN,100.000000,15.0,88.000000,NaN,1127.000000,31.000000,31.000000,NaN,NaN


In [10]:
#analyze data, get foods with only grades a, b, c, d, e
#get only their carb/protein/fat as preliminary analysis

only_a = a.loc[a['nutrition_grade_fr'] == 'a']
only_b = a.loc[a['nutrition_grade_fr'] == 'b']
only_c = a.loc[a['nutrition_grade_fr'] == 'c']
only_d = a.loc[a['nutrition_grade_fr'] == 'd']
only_e = a.loc[a['nutrition_grade_fr'] == 'e']

mac_only_a = only_a[['carbohydrates_100g', 'proteins_100g', 'fat_100g']]
mac_only_b = only_b[['carbohydrates_100g', 'proteins_100g', 'fat_100g']]
mac_only_c = only_c[['carbohydrates_100g', 'proteins_100g', 'fat_100g']]
mac_only_d = only_d[['carbohydrates_100g', 'proteins_100g', 'fat_100g']]
mac_only_e = only_e[['carbohydrates_100g', 'proteins_100g', 'fat_100g']]

In [11]:
mac_only_a.describe()

,carbohydrates_100g,proteins_100g,fat_100g
count,190.000000,211.000000,190.000000
mean,31.603895,8.586445,2.948316
std,29.210177,6.433398,3.421920
min,0.000000,0.000000,0.000000
25%,4.525000,2.900000,0.700000
50%,15.800000,7.800000,1.700000
75%,65.000000,12.500000,3.975000
max,83.600000,32.000000,23.900000


In [12]:
# ALEX: remove plotting, get notebook to run
# order = sorted(a.nutrition_grade_fr.unique()[1:])
# ax = sns.boxplot(x="nutrition_grade_fr", y="fat_100g", data=a, order=order)
_ = a.nutrition_grade_fr.unique()[1:]

In [13]:
#order = sorted(a.nutrition_grade_fr.unique()[1:])
# ALEX: remove plotting
# order = ["a", "b", "c", "d", "e"]
# ax = sns.boxplot(x="nutrition_grade_fr", y="sugars_100g", data=a, order=order)
# sns.plt.title('Sugar vs Grade')

In [14]:
# ALEX: remove plotting
# ax = sns.boxplot(x="nutrition_grade_fr", y="sugars_100g", data=a, order=order)
# sns.plt.title('Fat vs Grade')


In [15]:
# ALEX: remove plotting
# ax = sns.boxplot(x="nutrition_grade_fr", y="proteins_100g", data=a, order=order)
# sns.plt.title('Protein vs Grade')
